In [4]:
#!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

Mounted at /content/gdrive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
from fastbook import *
from IPython.display import display,HTML

In [6]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

In [7]:
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

In [8]:
txt = files[0].open().read(); txt[:75]

"This is one of t.v.'s greatest mini-series! It comes to life almost as well"

In [9]:
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks, 30))

(#55) ['This','is','one','of','t.v','.',"'s",'greatest','mini','-','series','!','It','comes','to','life','almost','as','well','as','the','book','did','.','Also','the','cast','was','outstanding','to'...]


In [10]:
first(spacy(['The U.S. dollar $1 is $1.00.']))

(#9) ['The','U.S.','dollar','$','1','is','$','1.00','.']

In [11]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

(#62) ['xxbos','xxmaj','this','is','one','of','t.v','.',"'s",'greatest','mini','-','series','!','xxmaj','it','comes','to','life','almost','as','well','as','the','book','did','.','xxmaj','also','the','cast'...]


In [12]:
defaults.text_proc_rules

[<function fastai.text.core.fix_html(x)>,
 <function fastai.text.core.replace_rep(t)>,
 <function fastai.text.core.replace_wrep(t)>,
 <function fastai.text.core.spec_add_spaces(t)>,
 <function fastai.text.core.rm_useless_spaces(t)>,
 <function fastai.text.core.replace_all_caps(t)>,
 <function fastai.text.core.replace_maj(t)>,
 <function fastai.text.core.lowercase(t, add_bos=True, add_eos=False)>]

In [13]:
coll_repr(tkn('©   Fast.ai www.fast.ai/INDEX'), 31)

"(#11) ['xxbos','©','xxmaj','fast.ai','xxrep','3','w','.fast.ai','/','xxup','index']"

In [14]:
txts = L(o.open().read() for o in files[:2000])

In [15]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

In [16]:
subword(1000)

"▁This ▁is ▁one ▁of ▁t . v . ' s ▁great est ▁ mi ni - s er ies ! ▁It ▁come s ▁to ▁life ▁almost ▁as ▁well ▁as ▁the ▁book ▁did . ▁Al s o ▁the ▁cast ▁was ▁out"

In [17]:
subword(200)

"▁ T h i s ▁is ▁ on e ▁of ▁ t . v . ' s ▁g re a t es t ▁ m in i - s er i es ! ▁I t ▁c o m es ▁to"

In [18]:
subword(10000)

"▁This ▁is ▁one ▁of ▁t . v . ' s ▁greatest ▁mini - series ! ▁It ▁come s ▁to ▁life ▁almost ▁as ▁well ▁as ▁the ▁book ▁did . ▁Also ▁the ▁cast ▁was ▁outstanding ▁to ▁play ▁the ▁roles . ▁I '"

In [19]:
toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

(#62) ['xxbos','xxmaj','this','is','one','of','t.v','.',"'s",'greatest','mini','-','series','!','xxmaj','it','comes','to','life','almost','as','well','as','the','book','did','.','xxmaj','also','the','cast'...]


In [20]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#62) ['xxbos','xxmaj','this','is','one','of','t.v','.',"'s",'greatest'...]

In [21]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)

"(#2192) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the',',','.','and','a','of','to','in','is','it','i'...]"

In [22]:
nums = num(toks)[:20]; nums

TensorText([   2,    8,   23,   17,   39,   14, 1615,   11,   22,  876,    0,   25,  185,   45,    8,   18,  321,   15,  100,  202])

In [23]:
' '.join(num.vocab[o] for o in nums)

"xxbos xxmaj this is one of t.v . 's greatest xxunk - series ! xxmaj it comes to life almost"

In [24]:
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
tokens = tkn(stream)
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter,",",we,will,go,back,over,the,example,of,classifying
movie,reviews,we,studied,in,chapter,1,and,dig,deeper,under,the,surface,.,xxmaj
first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into,numbers,and
how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have,another,example
of,the,preprocessor,used,in,the,data,block,xxup,api,.,\n,xxmaj,then,we
will,study,how,we,build,a,language,model,and,train,it,for,a,while,.


In [25]:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15:i*15+seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter
movie,reviews,we,studied,in
first,we,will,look,at
how,to,customize,it,.
of,the,preprocessor,used,in
will,study,how,we,build


In [26]:
#hide_input
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+seq_len:i*15+2*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

",",we,will,go,back
chapter,1,and,dig,deeper
the,processing,steps,necessary,to
xxmaj,by,doing,this,","
the,data,block,xxup,api
a,language,model,and,train


In [27]:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+10:i*15+15] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

over,the,example,of,classifying
under,the,surface,.,xxmaj
convert,text,into,numbers,and
we,'ll,have,another,example
.,\n,xxmaj,then,we
it,for,a,while,.


In [28]:
nums200 = toks200.map(num)

In [29]:
dl = LMDataLoader(nums200)

In [30]:
x,y = first(dl)
x.shape,y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [31]:
' '.join(num.vocab[o] for o in x[0][:20])

"xxbos xxmaj this is one of t.v . 's greatest xxunk - series ! xxmaj it comes to life almost"

In [32]:
' '.join(num.vocab[o] for o in y[0][:20])

"xxmaj this is one of t.v . 's greatest xxunk - series ! xxmaj it comes to life almost as"

In [33]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [34]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj i 've seen enough of both xxmaj little xxmaj richard in interviews and in performances and enough of poor xxmaj leon pigeonholed into these 50s / 60s musical bio pics to know that xxmaj leon was not the right actor for this role . xxmaj leon was so right as xxmaj david xxmaj xxunk in xxmaj the xxmaj temptations , but fails utterly to capture the essence of xxmaj little xxmaj richard in this film . \n\n xxmaj","xxmaj i 've seen enough of both xxmaj little xxmaj richard in interviews and in performances and enough of poor xxmaj leon pigeonholed into these 50s / 60s musical bio pics to know that xxmaj leon was not the right actor for this role . xxmaj leon was so right as xxmaj david xxmaj xxunk in xxmaj the xxmaj temptations , but fails utterly to capture the essence of xxmaj little xxmaj richard in this film . \n\n xxmaj actor"
1,") was the only xxmaj tarzan movie i did n't see when i was a kid . xxmaj it sounded boring . xxmaj now i have seen it . i have seen the ape xxunk ) about a hundred times and i keep a copy on my drive . xxmaj it 's a remarkable movie . xxmaj it 's almost flawless . xxmaj tarzan and his xxunk ) however , falters . xxmaj it 's not harmonic and it 's","was the only xxmaj tarzan movie i did n't see when i was a kid . xxmaj it sounded boring . xxmaj now i have seen it . i have seen the ape xxunk ) about a hundred times and i keep a copy on my drive . xxmaj it 's a remarkable movie . xxmaj it 's almost flawless . xxmaj tarzan and his xxunk ) however , falters . xxmaj it 's not harmonic and it 's parts"


In [35]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3,
    metrics=[accuracy, Perplexity()]).to_fp16()

In [36]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.011889,3.902912,0.300572,49.546524,22:40


In [37]:
learn.save('1epoch')

Path('/root/.fastai/data/imdb/models/1epoch.pth')

In [ ]:
learn = learn.load('1epoch')

In [44]:
learn.unfreeze()
learn.fit_one_cycle(5, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.759654,3.754122,0.317472,42.696716,23:42
1,3.670729,3.662239,0.328046,38.948463,23:27
2,3.548875,3.604893,0.335181,36.777748,24:02
3,3.454730,3.572276,0.339273,35.597527,23:44
4,3.375829,3.571942,0.339856,35.585629,23:57


In [45]:
learn.save_encoder('finetuned')

In [46]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75)
         for _ in range(N_SENTENCES)]
print("\n".join(preds))

i liked this movie because it 's wonderful ! i loved the story , the humor , and the movie . i had to watch it twice -- the second time i watched it , and it was a very good movie . i recommend
i liked this movie because i like Steve mcqueen , and i love Buddy Holly and Holly Hunter . Since i saw Lenny on a Sunday evening , i thought it was a great movie . i


In [48]:
TEXT = "I don't liked this movie but"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75)
         for _ in range(N_SENTENCES)]
print("\n".join(preds))

i do n't liked this movie but i saw it as a once - on - a - lifetime Lifetime movie . It was really a great movie . It gives you a put together , can be enjoyed by someone who lives with
i do n't liked this movie but i was pleasantly surprised . From the title , i thought that it was a good low budget film , but i had to go crazy to see it . It 's an ideal film for Shaq


In [49]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [50]:
dls_clas.show_batch(max_n=3)

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos * ! ! - xxup spoilers - ! ! * \n\n xxmaj before i begin this , let me say that i have had both the advantages of seeing this movie on the big screen and of having seen the "" authorized xxmaj version "" of this movie , remade by xxmaj stephen xxmaj king , himself , in 1997 . \n\n xxmaj both advantages made me appreciate this version of "" the xxmaj shining , "" all the more . \n\n xxmaj also , let me say that xxmaj i 've read xxmaj mr . xxmaj king 's book , "" the xxmaj shining "" on many occasions over the years , and while i love the book and am a huge fan of his work , xxmaj stanley xxmaj kubrick 's retelling of this story is far more compelling … and xxup scary . \n\n xxmaj kubrick",pos
2,"xxbos xxmaj some have praised xxunk xxmaj lost xxmaj xxunk as a xxmaj disney adventure for adults . i do n't think so -- at least not for thinking adults . \n\n xxmaj this script suggests a beginning as a live - action movie , that struck someone as the type of crap you can not sell to adults anymore . xxmaj the "" crack staff "" of many older adventure movies has been done well before , ( think xxmaj the xxmaj dirty xxmaj dozen ) but xxunk represents one of the worse films in that motif . xxmaj the characters are weak . xxmaj even the background that each member trots out seems stock and awkward at best . xxmaj an xxup md / xxmaj medicine xxmaj man , a tomboy mechanic whose father always wanted sons , if we have not at least seen these before ,",neg


In [51]:
nums_samp = toks200[:10].map(num)

In [52]:
nums_samp.map(len)

(#10) [62,230,186,692,161,257,336,315,927,164]

In [53]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5,
                                metrics=accuracy).to_fp16()

In [54]:
learn = learn.load_encoder('finetuned')

In [55]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.250231,0.191324,0.925920,01:23


In [56]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.226022,0.173293,0.933160,01:23


In [57]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.192721,0.155594,0.941520,01:39


In [58]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.162073,0.154343,0.942880,01:57
1,0.144849,0.154234,0.942600,01:55
